<hr style="height:0px; visibility:hidden;" />

<h1><center><b>GL4U: RNAseq<b></center></h1>

# Pipeline for Processing GeneLab RNA-sequencing Data

> **The GL4U RNAseq Jupyter Notebooks (JNs) are designed to teach students how to process RNA sequencing data using the [GeneLab standard pipeline](https://github.com/nasa/GeneLab_Data_Processing/blob/master/RNAseq/README.md). Below are step-by-step instructions for processing samples derived from the soleus (aka "anti-gravity") muscle of mice that were either flown in space aboard the International Space Station (ISS) (spaceflight, FLT) or kept in an environmental simulator on Earth to serve as ground control (GC) animals during NASA's Rodent Research - 1 mission. More information about the samples analyzed here can be found in the [Open Science Data Repository](https://osdr.nasa.gov/bio/repo) under [OSD-104](https://osdr.nasa.gov/bio/repo/data/studies/OSD-104).**  

---

### RNAseq Pipeline Overview
> The RNAseq Processing JNs, parts 1 and 2, will cover the pipeline steps outlined in red. For more information about how GeneLab processes bulk RNAseq data, visit the [GeneLab Data Processing GitHub repository](https://github.com/nasa/GeneLab_Data_Processing/tree/master/RNAseq).  

<br>

<div align="center">
<img src="https://raw.githubusercontent.com/nasa/GeneLab-Training/refs/heads/GL4U_RNAseq_2024_Colab/images/rnaseq-processing-pipeline.png" alt="RNAseq processing pipeline">
</div>

<br>

<hr style="height:0px; visibility:hidden;" />
    
<h1><center>2. RNAseq processing: alignment to counts</center></h1>

<div class="alert alert-block alert-success">
Here we are going to mount our google drive then pick up where we left off with the previous notebook, and continue following the steps in the GeneLab standard RNAseq processing pipeline to generate raw count data.
</div>

---

This is notebook 2 of 3 of GL4U's RNAseq Module Set. It is expected that the GL4U Introduction Module Set and the previous RNAseq module have been completed already.

---

## Set up the notebook environment
Run the following commands to mount your google drive then point to the location in your google drive folder containing all the files you downloaded and generated in the previous notebook.
> *For step-by-step instructions to mount your google drive, open the following link in a new browser tab: [How to Mount Google Drive](https://github.com/nasa/GeneLab-Training/blob/GL4U_Intro_2024_Colab/On-Demand/Mount_Google_Drive.md).*



In [7]:
# mount google drive
from google.colab import drive
drive.flush_and_unmount()
drive.mount("mnt")

Mounted at mnt


In [8]:
# define the location of the GL4U RNAseq directory in your Google Drive
import os
GL4U_RNASEQ_DIR="/content/mnt/MyDrive/NASA/GL4U/RNAseq"


---
<div class="alert alert-block alert-info">
<b>Note:</b> Google Colab does not currently offer the bash programming language. However, we are able to execute bash commands in the python programming language by adding characters such as a "!" or a "%" in front of the command as you will see throughout this JN.
</div>

<a id="setup"></a>

## 0. Setup

We'll start by moving to the location that contains the RNAseq files we will use in this JN:

In [9]:
%cd /content/mnt/MyDrive/NASA/GL4U/RNAseq/
!pwd

[Errno 2] No such file or directory: '/content/mnt/MyDrive/NASA/GL4U/RNAseq/'
/content
/content


<a class="anchor" id="paths"></a>

### Define Paths to GL4U: RNAseq Bootcamp Processed Data

Run the following commands to set up variables that define the paths to the files we will be using in this JN:


In [10]:
metadata="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/Metadata"
raw_fastq="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/00-RawData/Fastq"
raw_fastqc="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/00-RawData/FastQC_Reports"
raw_multiqc="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/00-RawData/FastQC_Reports/raw_multiqc_report"
trimmed_fastq="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/01-TG_Preproc/Fastq"
trimmed_fastqc="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/01-TG_Preproc/FastQC_Reports"
trimmed_multiqc="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/01-TG_Preproc/FastQC_Reports/trimmed_multiqc_report"
STAR_output="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/02-STAR_Alignment"
STAR_logs="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/02-STAR_Alignment/Log_files"
aligned_multiqc="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/02-STAR_Alignment/aligned_multiqc_report"
REFs="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/References"
RSeQC_infer_exp="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/RSeQC_analyses/infer_exp"
RSeQC_infer_exp_multiqc="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/RSeQC_analyses/infer_exp/infer_exp_multiqc_report"
RSEM_output="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/03-RSEM_Counts"
RSEM_logs="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/03-RSEM_Counts/Log_files"
count_multiqc="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/03-RSEM_Counts/count_multiqc_report"
processing="/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/processing_scripts"

Let's take another look at the file with information about the samples we will continue processing in the `Metadata` sub-directory using the `column` command to help with formatting the print-out (where `-s,` specifies the comma as the delimiter and `-t` tells it to try to make a table):


In [11]:
!column -s, -t $metadata/OSD-104_Sample_Metadata.csv

column: /content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/Metadata/OSD-104_Sample_Metadata.csv: No such file or directory


**Okay, now that we've reminded ourselves of the flight and ground control samples we are working with, it's time to continue processing the RNAseq data!**

<div class="alert alert-block alert-info">
<b>Note:</b><br>

*As a reminder, for the purposes of this bootcamp we will only be running one sample, FLT_M23, through each step of the pipeline.*

</div>

---

<a class="anchor" id="starindex"></a>

## 1. Build a STAR Index for the Reference Genome

Now that our RNA sequence data is of high quality, our next step is to identify which gene(s) each RNA sequence came from. To do this, we will align the reads back to the reference genome of the organism from which the samples were derived using the [STAR (Spliced Transcripts Alignment to a Reference)](https://github.com/alexdobin/STAR) aligner.

Since our samples came from the soleus muscle of mice (scientific name _Mus musculus_), we will align the trimmed reads to the _Mus musculus_ reference genome (GRCm39), file `Mus_musculus.GRCm39.dna.primary_assembly.fa` from the ensembl database found [here](http://ftp.ensembl.org/pub/release-112/fasta/mus_musculus/dna/). In addition to the reference genome, we will also need to provide STAR with information about the gene features of all annotated genes in the reference genome. (_Note: Annotated genes are genes whose genomic coordinates and function, i.e. where they are and what they do, are known._) All of the gene feature information STAR needs is provided in the `Mus_musculus.GRCm39.112.gtf` gene transfer format (GTF) file associated with the _Mus musculus_ reference genome (GRCm39) from the ensembl database found [here](http://ftp.ensembl.org/pub/release-112/gtf/mus_musculus/).

> For more information about the ensembl gene transfer format (GTF) and gene feature format (GFF) files, click [here](http://m.ensembl.org/info/website/upload/gff.html).

Before we can align our trimmed reads to the _Mus musculus_ reference genome, the STAR program needs to organize all the information provided in the genome and GTF files to be able to accurately assign each read to the gene from which it came. To do this, we will first run STAR in the `genomeGenerate` mode, which will create a STAR index containing the genome reference and gene feature information in the STAR format.

<div class="alert alert-block alert-info">
<b>Note:</b><br>

Creating a STAR index requires a lot of compute and storage resources and can take a while to complete, so we did it for you. You're welcome :0)

</div>

The following command was used to generate the STAR index for _Mus musculus_:

```bash
STAR --runThreadN 16 \
 --runMode genomeGenerate \
 --limitGenomeGenerateRAM 70000000000 \
 --genomeDir STAR_index \
 --genomeFastaFiles Mus_musculus.GRCm39.dna.primary_assembly.fa \
 --sjdbGTFfile $genome_gtf/Mus_musculus.GRCm39.112.gtf \
 --sjdbOverhang 99
```


<div class="alert alert-block alert-info">
<b><i>Code Breakdown</i></b>
    
<br>

**Parameter Definitions:**

* `--runThreadN` – number of threads available to create the STAR indexed reference
* `--runMode` - instructs STAR to run genome indices generation job to create the STAR indexed reference
* `--limitGenomeGenerateRAM` - maximum RAM available (in bytes) to generate STAR reference
  > _Note: At least 35GB are needed for mouse and the example above shows 70GB_
* `--genomeDir` - specifies the directory where the STAR indexed reference will be stored
  > _Note: At least 100GB of available disk space is required for mammalian genomes_
* `--genomeFastaFiles` - specifies the uncompressed fasta file containing the genome reference sequences
* `--sjdbGTFfile` – specifies the uncompressed file containing annotated transcripts in the standard GTF format
* `--sjdbOverhang` - indicates the length of the genomic sequence around the annotated junction to be used in constructing the splice junctions database
  > _Note: The length should be one less than the length of the (longest) read_

**Input Data:**
- `*.fasta` (reference genome sequence)
- `*.gtf` (reference genome transfer feature file)

**Output Data:**

STAR indexed genome reference, which consists of the following files:
- `chrLength.txt`
- `chrNameLength.txt`
- `chrName.txt`
- `chrStart.txt`
- `exonGeTrInfo.tab`
- `exonInfo.tab`
- `geneInfo.tab`
- `Genome`
- `genomeParameters.txt`
- `SA`
- `SAindex`
- `sjdbInfo.txt`
- `sjdbList.fromGTF.out.tab`
- `sjdbList.out.tab`
- `transcriptInfo.tab`

</div>

<br>

---

<a class="anchor" id="star"></a>

## 2. Align Reads to the Reference Genome

<a class="anchor" id="staralign"></a>

### 2a. Align Trimmed Sequence Data with STAR

We are now ready to align the trimmed reads to the mouse reference genome. This step of determining the genomic location (and ultimately, gene) from which each short sequence was derived is quite challenging. Any ideas why that is?

**Input your response to the question in the text box below:**

<br>

<div class="alert alert-block alert-success">

<details>
<summary><b>Answer</b></summary>

<br>

- Reads are very short relative to transcripts, and some genomes (including mouse and human) can be quite large and contain lots of non-unique sequences such as pseudogenes, which makes mapping difficult.
- Aligners also have to cope with mismatches and indels (insertions and deletions) caused by genomic variation and sequencing errors.
- Many organisms have introns in their genes and therefore, reads align non-contiguously (not touching). Placing spliced reads across introns and determining exon-intron boundaries is difficult, especially considering that introns can be thousands of basepairs long.
    
</details>
</div>

<br>

Spliced aligners, including [STAR](https://github.com/alexdobin/STAR), were developed to more accurately identify non-contiguously aligned reads. Additionally, the STAR spliced alignment program is able to perform an unbiased search for splice junctions without needing any prior information on location, sequence signals, or intron length. It is also capable of aligning a read with multiple splice junctions, indels, and mismatches and those with low-quality ends.

In order to improve the detection and quantification of splice sites, we will run STAR in the “two-pass mode”. Here, splice sites are detected in the initial mapping to the reference and used to build a new reference that includes these splice sites. Reads are then re-mapped to this dynamically generated reference to improve the quantification of splice isoforms.

<div class="alert alert-block alert-info">
<b>Note:</b><br>

Aligning trimmed reads with STAR requires a lot of compute and storage resources and can take a while to complete, so we did it for you. You're welcome :0)

</div>


The following `STAR` command was used to align trimmed reads for all 12 samples to the _Mus musculus_ reference:

```bash
STAR --twopassMode Basic \
 --limitBAMsortRAM 55000000000 \
 --genomeDir STAR_index \
 --outSAMunmapped Within \
 --outFilterType BySJout \
 --outSAMattributes NH HI AS NM MD MC \
 --outFilterMultimapNmax 20 \
 --outFilterMismatchNmax 999 \
 --outFilterMismatchNoverReadLmax 0.04 \
 --alignIntronMin 20 \
 --alignIntronMax 1000000 \
 --alignMatesGapMax 1000000 \
 --alignSJoverhangMin 8 \
 --alignSJDBoverhangMin 1 \
 --sjdbScore 1 \
 --readFilesCommand zcat \
 --runThreadN 18 \
 --outSAMtype BAM SortedByCoordinate \
 --quantMode TranscriptomeSAM GeneCounts \
 --outSAMheaderHD @HD VN:1.4 SO:coordinate \
 --outFileNamePrefix $STAR_output/${sample}/${sample}_ \
 --readFilesIn $trimmed_fastq/${sample}_R1_trimmed.fastq.gz $trimmed_fastq/${sample}_R2_trimmed.fastq.gz
```

<div class="alert alert-block alert-info">
<b><i>Code Breakdown</i></b>
    
<br>

**Parameter Definitions:**

* `--twopassMode` – specifies 2-pass mapping mode; the `Basic` option instructs STAR to perform the 1st pass mapping, then automatically extract junctions, insert them into the genome index, and re-map all reads in the 2nd mapping pass
* `--limitBAMsortRAM` - maximum RAM available (in bytes) to sort the bam files, the example above indicates 55GB
* `--genomeDir` - specifies the path to the directory where the STAR indexed reference is stored
* `--outSAMunmapped` - specifies output of unmapped reads in the sam format; the `Within` option instructs STAR to output the unmapped reads within the main sam file
* `--outFilterType` - specifies the type of filtering; the `BySJout` option instructs STAR to keep only those reads that contain junctions that passed filtering in the SJ.out.tab output file
* `--outSAMattributes` - list of desired sam attributes in the order desired for the output sam file
  > _Note: SAM attribute descriptions can be found [here](https://samtools.github.io/hts-specs/SAMtags.pdf)_
* `--outFilterMultimapNmax` – specifies the maximum number of loci the read is allowed to map to; all alignments will be output only if the read maps to no more loci than this value
* `--outFilterMismatchNmax` - maximum number of mismatches allowed to be included in the alignment output
* `--outFilterMismatchNoverReadLmax` - ratio of mismatches to read length allowed to be included in the alignment output; the `0.04` value indicates that up to 4 mismatches are allowed per 100 bases
* `--alignIntronMin` - minimum intron size; a genomic gap is considered an intron if its length is equal to or greater than this value, otherwise it is considered a deletion
* `--alignIntronMax` - maximum intron size
* `--alignMatesGapMax` - maximum genomic distance (in bases) between two mates of paired-end reads
  > _Note: This option should be removed for single-end reads_
* `--alignSJoverhangMin` - minimum overhang (i.e. block size) for unannotated spliced alignments
* `--alignSJDBoverhangMin` - minimum overhang (i.e. block size) for annotated spliced alignments
* `--sjdbScore` - additional alignment score for alignments that cross database junctions
* `--readFilesCommand` - specifies command needed to interpret input files; the `zcat` option indicates input files are compressed with gzip and zcat will be used to uncompress the gzipped input files
* `--runThreadN` - indicates the number of threads to be used for STAR alignment
* `--outSAMtype` - specifies desired output format; the `BAM SortedByCoordinate` options specify that the output file will be sorted by coordinate and be in the bam format
* `--quantMode` - specifies the type(s) of quantification desired; the `TranscriptomeSAM` option instructs STAR to output a separate sam/bam file containing alignments to the transcriptome and the `GeneCounts` option instructs STAR to output a tab delimited file containing the number of reads per gene
* `--outSAMheaderHD` - indicates a header line for the sam/bam file
* `--outFileNamePrefix` - specifies the path to and prefix for the output file names; for GeneLab the prefix is the sample id
* `--readFilesIn` - path to input read 1 (forward read) and read 2 (reverse read); for paired-end reads, read 1 and read 2 should be separated by a space; for single-end reads only read 1 should be indicated


**Input Data:**
- `STAR indexed reference genome` (output from [Step 1](#3.-Build-a-STAR-Index-for-the-Reference-Genome) above)
- `*trimmed.fastq.gz` (trimmed reads, output from Step 2a of the 01-RNAseq_processing_1of2_colab JN)

**Output Data:**

> _Note: Detailed descriptions of all STAR output files can be found in the [STAR manual](https://github.com/alexdobin/STAR/blob/master/doc/STARmanual.pdf) under the "Output files" section._

- `*Aligned.sortedByCoord.out.bam` (binary, alignment map containing sorted reads mapping to genome)
- `*Aligned.toTranscriptome.out.bam` (binary, alignment map containing sorted reads mapping to transcriptome)
- `*Log.final.out` (log file containing a summary of the mapping info/stats such as # reads mapped, etc)
- `*ReadsPerGene.out.tab` (tab-delimited file containing STAR read counts per gene with 4 columns that correspond to different strandedness options: column 1 = gene ID, column 2 = counts for unstranded RNAseq, column 3 = counts for 1st read strand aligned with RNA, column 4 = counts for 2nd read strand aligned with RNA)
- `*Log.out` (log file containing detailed information about the mapping run, useful for troubleshooting)
- `*Log.progress.out` (file containing the job progress statistics that is updated every minute during the mapping run)
- `*SJ.out.tab` (tab-delimited file containing identified splice junctions and respective count data)
- `*_STARgenome` (directory containing the extracted junctions from the 1st pass mapping that was inserted into the genome index)
  - `sjdbInfo.txt`
  - `sjdbList.out.tab`
- `*_STARpass1` (directory containing splice junction and summary mapping info/stats from the 1st pass mapping)
  - `Log.final.out`
  - `SJ.out.tab`
- `*_STARtmp` (directory containing subdirectories that are empty – this was the location for temp files that were automatically removed after successful completion)
  - `BAMsort`

</div>

<br>___________

Let's take a look at the first few lines of the `*Aligned.toTranscriptome.out.bam` output file from sample FLT_M23 using samtools.

<div class="alert alert-block alert-info">
<b>Note:</b><br>

BAM files are compressed, binary versions of Sequence Alignment/Map (SAM) files and need to be uncompressed using `samtools view` to view the file contents.

</div>Before we can run samtools, we first need to install it by running the following command:

In [12]:
# install samtools
!sudo apt-get install samtools > /dev/null

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


We're now ready to view the first few lines of the `*Aligned.toTranscriptome.out.bam` output file from sample FLT_M23 using samtools.

In [15]:
!samtools view $STAR_output/FLT_M23/FLT_M23_Aligned.toTranscriptome.out.bam | head

J00113:162:H7W32BBXX:1:1101:11261:1947	163	ENSMUST00000029076	1240	255	99M	=	1404	264	CACACGTTAACATCATTGTAGATCTCAAAAACTGAATTCATAATGTGTTTTACTTCAAATAATACCAGCGATAATCCATCACTCGTTAAAAATTTGCCT	AAFFFJ<FF<FJJJFJJ<<<<<<<<FFFJJJJJF<<7FJFJJAJA<A<7AFFJJJJJ<FJJAJAJJJFJJAFAAAJFJAFAJFJAFA-A-A<FFJJJJJ	NH:i:1	HI:i:1	MC:Z:100M
J00113:162:H7W32BBXX:1:1101:11261:1947	83	ENSMUST00000029076	1404	255	100M	=	1240	-264	GTTTGATCCAGCATTTAAATTTCTTCTTCATAAAGATGGTTTTCTTTGCCCAAAGTAGAGCCATTTATTTTTTATTTCACTACTTTAATCTTTGCATGCC	FJJFFJAFJJJJ<JJJJJJJJJJFJJJJJFFF<<FFAJJJJJJJJFFFJJF<<7F<77-JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFFJJFFFAA	NH:i:1	HI:i:1	MC:Z:99M
J00113:162:H7W32BBXX:1:1101:11211:1965	163	ENSMUST00000034453	812	255	100M	=	948	236	GGCAGGTCATCACCATCGGCAATGAGCGTTTCCGTTGCCCGGAGACGCTCTTCCAGCCTTCCTTTACCGGTATGGAGTCTGCGGGGATCCATGAGACCAC	AAFFFJ777FFJJJJ-F7FA7-<<<F<F--<<FF--AFJJJF7A-<F<-AA7AAFJ7JA7AF<<FA-77A7--7A-7-<A-77<FF)77A--A-<-AA-A	NH:i:1	HI:i:1	MC:Z:100M
J00113:162:H7W32BBXX:1:1101:11211:1965	83	ENSMUST00000034453	948	25

<div class="alert alert-block alert-info">
<b>SAM file content</b><br>

All BAM/SAM files contain at least the following 11 tab-separated fields in order of column location:

1) **QNAME** - Query template name: Info about the sequencing run that generated the read, found in line 1 of the trimmed fastq file.

2) **FLAG** - bitwise FLAG: Information about the alignment encoded in bits. To easily decode the SAM FLAG, type it into the Broad Institute's SAM FLAG decoder [here](https://broadinstitute.github.io/picard/explain-flags.html)

3) **RNAME** - Reference sequence name: Name of the reference sequence the read aligned to (this will be the ensembl transcript ID in the transcript-aligned BAM file as shown above, and the chromosome number in the genome-aligned BAM file)

4) **POS** - 1-based leftmost mapping position: The position on the reference genome in which the left most base of the read aligns.

5) **MAPQ** - Mapping quality: Equality to the -10log(base10) of the probability that the mapping position is wrong; a value of 255 indicates that the mapping quality is not available.

6) **CIGAR** - CIGAR string: Aligned read length and associated operation, which encodes information about the alignment relative to the reference (i.e. match/mismatch, insertion/deletion).

7) **RNEXT** - Reference name of the mate/next read: Reference sequence name of the next aligned read in the template, if it's the same, this is represented with an equal (=) sign.

8) **PNEXT** - Position of the mate/next read: 1-based position of the next aligned read in the template.

9) **TLEN** - Observed template length: Length from the leftmost position of read 1 to the rightmost position of read 2 for aligned paired-end sequence data.

10) **SEQ** - Segment sequence: Sequence of the aligned trimmed read, found in line 2 of the trimmed fastq file.

11) **QUAL** - ASCII of Phred-scaled base quality +33: Base call quality scores, found in line 4 of the trimmed fastq file.

12) Additional SAM attributes that were added with the `--outSAMattributes` option in the STAR alignment command above.

> _Detailed descriptions of the Sequence Alignment/Map (SAM) format specification can be found [here](https://samtools.github.io/hts-specs/SAMv1.pdf)._

</div>

<br>

**Now that you know what the fields mean, look at the BAM file above and answer the following questions:**

1. How many transcripts are represented in the first 10 lines of the BAM file? How do you know?

2. How many read pairs are shown? How do you know?

3. How many reads align in the same orientation relative to the reference? What about in the opposite orientation? How to you know?

4. What are the min and max template length shown? How do you know?

**Input your responses to each question in the text boxes below:**
> *Answer each question in your own words BEFORE checking your answers against the solutions below. Please do not copy and paste the answers provided.*


Question 1:
5, transcripts are called
ENSMUST00000029076 ENSMUST00000034453 ENSMUST00000107038 ENSMUST00000175751 ENSMUST00000045243 in the third column

---



Question 2:
3, because now we are talking about read pairs and we have 3 couples: the first two entries are the first read pair, the second two entries are the second read pair and the last six entries are three couples for the third read pair

Question 3: five and five because in the TLEN column 5 lines have opposite sign, for a total of 5 + and 5 - reads.



Question 4: 138 and 264 are respectively the min and max template length; it can be known by looking at the TLEN column

<br>

<div class="alert alert-block alert-success">

<details>
<summary><b>Q1 Solution</b></summary>

<br>

There are 5 transcripts represented based on the number of unique reference sequence names (RNAME) displayed:

ENSMUST00000029076
ENSMUST00000034453
ENSMUST00000107038
ENSMUST00000175751
ENSMUST00000045243

</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q2 Solution</b></summary>

<br>

3 read pairs are shown. There's a unique set of sequences (mate pairs) for the first two transcripts but the same set of sequences (mate pairs) for the last three transcripts.
    
</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q3 Solution</b></summary>

<br>

5 reads align in the same orientation relative to the reference (positive TLEN value).
5 reads align in the opposite orientation relative to the reference (negative TLEN value).

</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q4 Solution</b></summary>

<br>

Min template length shown: 138 bases
Max template length shown: 264 bases

This is known by looking at the TLEN column.

</details>
</div>

---

Now let's take a look at the `*Log.final.out` file to assess the trimmed read alignment in sample FLT_M23 by running the following command:

In [16]:
!cat $STAR_output/FLT_M23/FLT_M23_Log.final.out

                                 Started job on |	Sep 17 18:36:53
                             Started mapping on |	Sep 17 18:44:01
                                    Finished on |	Sep 17 18:44:15
       Mapping speed, Million of reads per hour |	25.69

                          Number of input reads |	99911
                      Average input read length |	197
                                    UNIQUE READS:
                   Uniquely mapped reads number |	76999
                        Uniquely mapped reads % |	77.07%
                          Average mapped length |	196.96
                       Number of splices: Total |	54969
            Number of splices: Annotated (sjdb) |	54969
                       Number of splices: GT/AG |	54596
                       Number of splices: GC/AG |	335
                       Number of splices: AT/AC |	27
               Number of splices: Non-canonical |	11
                      Mismatch rate per base, % |	0.27%
                         Deleti

**Use the alignment log file above to answer the following questions:**

1. How many reads mapped to a unique location on the reference genome?

2. How many reads mapped to multiple locations on the reference genome?

3. Were any splice regions identified? If so, how many in total?

4. How many of the spliced regions were annotated? How many were non-canonical?

**Input your responses to each question in the text boxes below:**
> *Answer each question in your own words BEFORE checking your answers against the solutions below. Please do not copy and paste the answers provided.*


Question 1: looking at the file, we can say
Uniquely mapped reads number |	76999

Question 2:
looking at the file, we can say that Number of reads mapped to multiple loci |	11359

Question 3:
yes, by summing the annotated and non-canonical spliced regions we get a total of  Number of splices: Annotated |	54969 + non-canonical: 11, or 54980

Question 4:


<br>

<div class="alert alert-block alert-success">

<details>
<summary><b>Q1 Solution</b></summary>

<br>

There are 76,999 (77.07%) uniquely mapped reads.

</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q2 Solution</b></summary>

<br>

There are 11,359 (11.37%) multi-mapped reads.
> _Note: The are also an additional 103 (0.10%) multi-mapped reads, but they map to more loci than the threshold, defined by the `--outFilterMultimapNmax` parameter in the `STAR` command above, allows._
    
</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q3 Solution</b></summary>

<br>

Yes, 54,980 total spliced regions were identified (54,969 annotated + 11 non-canonical).

</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q4 Solution</b></summary>

<br>

Number of annotated spliced regions: 54,969
Number of non-canonical spliced regions: 11

</details>
</div>


---

<a class="anchor" id="alignmultiqc"></a>

### 2b. Compile Alignment Log Files with MultiQC

Rather than viewing the `Log.final.out` file for all 12 aligned samples individually, we'll again use the [MultiQC](https://docs.seqera.io/multiqc) program to compile the summary alignment info/stats from all samples and generate a single report (and associated data files). This will allow us to view the alignment statistics of all samples at once.

Before we can run MultiQC, we first need to install it by running the following commands:

In [17]:
# install MultiQC
%pip install --quiet multiqc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.7 MB/s eta 0:00:00


In [18]:
# check the MultiQC version
!multiqc --version

multiqc, version 1.33


We're now ready to compile the data in the `*Log.final.out` files from our 12 samples using MultiQC by running the following command:

In [19]:
!multiqc --interactive -n align_multiqc -o $aligned_multiqc $STAR_logs/


/// ]8;id=791671;https://multiqc.info\MultiQC]8;;\ 🎄 v1.33

       file_search | Search path: /content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/02-STAR_Alignment/Log_files
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 12/12  
              star | Found 12 reports
     write_results | Data        : mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/02-STAR_Alignment/aligned_multiqc_report/align_multiqc_data
     write_results | Report      : mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/02-STAR_Alignment/aligned_multiqc_report/align_multiqc.html
           multiqc | MultiQC complete


<div class="alert alert-block alert-info">
<b><i>Code Breakdown</i></b>
    
<br>

**Parameter Definitions:**

* `--interactive` – force reports to use interactive plots
* `-n` – prefix name for the output files
* `-o` – the output directory to store the MultiQC results
* `$STAR_logs/` – the directory holding the summary alignment info/stats data from each sample, provided as a positional argument

**Input Data:**
- `*Log.final.out` (summary alignment info/stats data)

**Output Data:**
- `align_multiqc_data` (directory containing the compiled summary alignment info/stats data from each sample, used to create the MultiQC report)
- `align_multiqc.html` (MultiQC report: an interactive webpage file containing graphical representations of the compiled alignment info/stats data)

</div>

<br>MultiQC is finished running when the loading animation on the left end of the code cell stops and a green checkmark appears. In a standard bash shell environment, if the multiQC job finishes successfully, you will see a "MultiQC complete" statement at the end of the standard output, and could check that the number of reports found matches what you expect.
> Note: There is only one `*Log.final.out` file generated per sample (for paired- or single-end data), so we expect one report per sample. Since we have 12 samples, MultiQC should detect 12 STAR reports.

Let's list the output files that were generated:


In [20]:
!ls -1 $aligned_multiqc/*

/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/02-STAR_Alignment/aligned_multiqc_report/align_multiqc.html

/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/02-STAR_Alignment/aligned_multiqc_report/align_multiqc_data:
llms-full.txt
multiqc_citations.txt
multiqc_data.json
multiqc_general_stats.txt
multiqc.log
multiqc.parquet
multiqc_sources.txt
multiqc_star.txt
star_alignment_plot.txt
star_summary_table.txt


Open the `aligned_multiqc_report` directory in the side panel of this Colab notebook by clicking: `/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/02-STAR_Alignment/aligned_multiqc_report`, as shown below.
> *Note: These files are stored in your Google Drive folder.*

<br>

<div align="center">
<img src="https://raw.githubusercontent.com/nasa/GeneLab-Training/refs/heads/GL4U_RNAseq_2024_Colab/images/gd_align_multiqc_dir.png" alt="Open the aligned multiQC report directory">
</div>
<br>

Click the 3 dots next to the `align_multiqc.html` file in the left side panel to download it to your computer as shown below.

<br>

<div align="center">
<img src="https://raw.githubusercontent.com/nasa/GeneLab-Training/refs/heads/GL4U_RNAseq_2024_Colab/images/gd_dl_align_multiqc.png" alt="Download aligned multiqc HTML file">
</div>
<br>

Navigate to the HTML file you downloaded on your computer then double click on it to open in your web browser to view the compiled alignment log info for all 12 samples, which should look similar to the following:

<br>

<div align="center">
<img src="https://raw.githubusercontent.com/nasa/GeneLab-Training/refs/heads/GL4U_RNAseq_2024_Colab/images/gd_open_align_multiqc.png" alt="Open aligned multiQC report">
</div>
<br>
<div class="alert alert-block alert-info">
<b>Note:</b><br>
    
Click [here](https://docs.seqera.io/multiqc/modules/star) for a description of the MultiQC plots derived from STAR data.

</div>

<div class="alert alert-block alert-warning">
    
Be sure to click the "Trust HTML" button in the top left corner of the MultiQC report to be able to see all the plots.

If the plots are still not visible, make sure JavaScript is enabled on the web-browser you are using.

</div>

**Take a look at the multiQC report of the alignment data above and answer the following questions:**

1. What are the min and max % uniquely aligned reads for all samples?

2. What are the min and max % of aligned reads that map to multiple locations?

**Input your responses to each question in the text boxes below:**
> *Answer each question in your own words BEFORE checking your answers against the solutions below. Please do not copy and paste the answers provided.*


Question 1: in the first table under general stat we can see that 80.6% of FLT_M25 is the max, while min of aligned reads in % is for GC_M38 and _37

Question 2: by looking at the alignment scores we can see that

the min of aligned reads that map to multiple locations is 9965 (FLT_M25), total reads are 100k around, so it's 9.96%

same argument for the max, 12.7% (GC_M36)


<br>

<div class="alert alert-block alert-success">

<details>
<summary><b>Q1 Solution</b></summary>

<br>

Min % uniquely aligned reads: 76.0% (GC_M37 and GC_M38)
Max % uniquely aligned reads: 80.6% (FLT_M25)

</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q2 Solution</b></summary>

<br>

Min % reads mapping to multiple loci: 10.0% (FLT_M25)
Max % reads mapping to multiple loci: 12.7% (GC_M36)

</details>
</div>


---

<a class="anchor" id="index"></a>

### 2c. Index Genome Aligned Reads

To prepare the genome aligned reads for strandedness assessment, the aligned reads must be indexed to allow the RSeQC program to locate particular regions of interest. We will use the [samtools](http://www.htslib.org/doc/samtools.html) program to index the genome aligned reads.

> Click on the link to learn more about the [samtools index program](http://www.htslib.org/doc/samtools-index.html) specifically.

Let's index the `*Aligned.sortedByCoord.out.bam` file for sample FLT_M23 using samtools by running the following command:


In [21]:
!samtools index -@ 2 \
 $STAR_output/FLT_M23/FLT_M23_Aligned.sortedByCoord.out.bam

<div class="alert alert-block alert-info">
<b><i>Code Breakdown</i></b>
    
<br>

**Parameter Definitions:**

* `-@` – indicates the number of threads to be used for samtools indexing
* `$STAR_output/${sample}/${sample}_Aligned.sortedByCoord.out.bam` – path to the `*Aligned.sortedByCoord.out.bam` file(s) you want to index, provided as a positional argument

**Input Data:**
- `*Aligned.sortedByCoord.out.bam` (binary, alignment map containing reads mapping to genome, sorted by coordinate, output from [Step 2a](#2a.-Align-Trimmed-Sequence-Data-with-STAR))

**Output Data:**
- `*Aligned.sortedByCoord.out.bam.bai` (index files for the respective `*Aligned.sortedByCoord.out.bam` files)

</div>

<br>___________

Let's list the output files that were generated:

In [22]:
!ls -1 $STAR_output/FLT_M23/

FLT_M23_Aligned.sortedByCoord.out.bam
FLT_M23_Aligned.sortedByCoord.out.bam.bai
FLT_M23_Aligned.toTranscriptome.out.bam
FLT_M23_Log.final.out
FLT_M23_Log.out
FLT_M23_Log.progress.out
FLT_M23_ReadsPerGene.out.tab
FLT_M23_SJ.out.tab
FLT_M23__STARgenome
FLT_M23__STARpass1


Note that the `*bai` file has been generated.

<div class="alert alert-block alert-info">
<b>Note:</b><br>
    
To preserve compute resources, the `*Aligned.sortedByCoord.out.bam` file has been indexed for all OSD-104 FLT and GC samples.

</div>

<br>

---

<a class="anchor" id="createbed"></a>

## 3. Create a BED File for the Reference Genome

Prior to quantitating our alignment data, we need to determine if/how reads were stranded during library preparation and sequencing. To do this, we will compare the strandedness of reads with the strandedness of transcripts using the [RSeQC Infer Experiment](http://rseqc.sourceforge.net/#infer-experiment-py) program.

<div class="alert alert-block alert-info">
<b>Note:</b><br>
    
The **strandedness of reads** is determined from the alignment data and the **strandedness of transcripts** is determined from the reference genome gene annotations.

</div>

Since our samples came from the soleus muscle of mice (scientific name _Mus musculus_), we will use the gene transfer format (GTF) file associated with the _Mus musculus_ reference genome (GRCm39) from the ensembl database found [here](ftp://ftp.ensembl.org/pub/release-112/gtf/mus_musculus/Mus_musculus.GRCm39.112.gtf.gz) to identify strandedness of transcripts. However, the RSeQC Infer Experiment program requires the annotated transcripts to be in the Browser Extensible Data (BED) format. Thus, before we can assess read strandedness relative to the _Mus musculus_ reference transcripts, we will first convert the mouse GTF file to a BED file using a two-step process:
> Step 1: Convert GTF to a GenePred table format using [gtfToGenePred](https://bioconda.github.io/recipes/ucsc-gtftogenepred/README.html)
>
> Step 2: Convert GenePred to BED format using [genePredToBed](https://bioconda.github.io/recipes/ucsc-genepredtobed/README.html)
>
> Click on the respective file links for more information about the [GenePred table format](https://genome.ucsc.edu/FAQ/FAQformat.html#format9) and the [BED file format](https://genome.ucsc.edu/FAQ/FAQformat.html#format1) from UCSC.

<div class="alert alert-block alert-info">
<b>Note:</b><br>

The BED file only needs to be created once, so we did it for you. You're welcome :0)

</div>


The following commands were used to create the BED file for _Mus musculus_:

**Step 1 - GTF to GenePred:**

```bash
gtfToGenePred Mus_musculus.GRCm39.112.gtf \
 Mus_musculus.GRCm39.112.gtf.genePred
```

<div class="alert alert-block alert-info">
<b><i>Code Breakdown</i></b>
    
<br>


**Parameter Definitions:**

* `Mus_musculus.GRCm39.112.gtf` – specifies the uncompressed file containing annotated transcripts in the standard GTF format, provided as a positional argument
* `Mus_musculus.GRCm39.112.gtf.genePred` – specifies the path to and the file name for the output GenePred file, provided as a positional argument

</div>


**Step 2 - GenePred to BED:**

```bash
genePredToBed Mus_musculus.GRCm39.112.gtf.genePred \
 Mus_musculus.GRCm39.112.gtf.bed
```

<div class="alert alert-block alert-info">
<b><i>Code Breakdown</i></b>
    
<br>

**Parameter Definitions:**

* `Mus_musculus.GRCm39.112.gtf.genePred` – specifies the file containing annotated transcripts in the GenePred format, provided as a positional argument
* `Mus_musculus.GRCm39.112.gtf.bed` – specifies the path to and the file name for the output BED file, provided as a positional argument

</div>

<br>

<div class="alert alert-block alert-info">
    

**Input Data:**
- `*.gtf` (genome transfer feature file)

**Output Data:**
- `*.genePred` (GenePred table format file)
- `*.bed` (Browser Extensible Data file)

</div>

<br>

---

<a class="anchor" id="strandedness"></a>

## 4. Determine Read Strandedness

<a class="anchor" id="inferexp"></a>

### 4a. Evaluate Strandedness with RSeQC Infer Experiment

Now we're ready to determine if/how reads were stranded during library preparation and sequencing. Why is this important to know prior to quantitation?

**Input your response to the question in the text box below:**

Depending on the fact that library is stranded or unstranded, it changes the information that we have in our hand: if the strandedness is not lost, then we know wheter the read should be assigned to the + strand or - strand and we can avoid misinterpretation of data


<br>

<div class="alert alert-block alert-success">

<details>
<summary><b>Answer</b></summary>

<br>

- Stranded RNA sequencing preserves strand information of a read, which helps to resolve read ambiguity in overlapping transcripts transcribed from complimentary DNA strands.

- This information is required for RSEM to properly assign reads to the correct gene of origin.
    
</details>
</div>

To do this, we will compare the strandedness of reads with the strandedness of transcripts using the [RSeQC Infer Experiment program](http://rseqc.sourceforge.net/#infer-experiment-py).

<div class="alert alert-block alert-info">
<b>Note:</b><br>
    
- [RSeQC](http://rseqc.sourceforge.net/) is an RNAseq quality control package containing several programs used to evaluate high throughput RNA sequence data. In addition to the Infer Experiment module used in this step, the current GeneLab RNAseq pipeline also utilizes the following modules to generate RNAseq QC metrics:
    - [RSeQC geneBody_coverage](http://rseqc.sourceforge.net/#genebody-coverage-py): Used to determine RNAseq reads coverage over the length of the transcripts
    - [RSeQC read_distribution](http://rseqc.sourceforge.net/#read-distribution-py): Used to quantify how aligned reads were distributed over genome features
    - [RSeQC inner_distance](http://rseqc.sourceforge.net/#inner-distance-py): Used to determine the insert size between two paired reads (distance from the end of R1 to the start of R2)

</div>

Let's evaluate read strandedness for sample FLT_M23 using the RSeQC Infer Experiment module:
Before we can run RSeQC, we first need to install it by running the following commands:


In [23]:
# install RSeQC
%pip install rseqc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.1/151.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.1/187.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 31.2 MB/s eta 0:00:00


In [24]:
# check RSeQC Infer Experiment version
!infer_experiment.py --version

infer_experiment.py 5.0.4


We're now ready to evaluate read strandedness for sample FLT_M23 using the RSeQC Infer Experiment module.


In [25]:
!infer_experiment.py -r $REFs/Mus_musculus.GRCm39.112.gtf.bed \
 -i $STAR_output/FLT_M23/FLT_M23_Aligned.sortedByCoord.out.bam \
 -s 15000000 \
 > $RSeQC_infer_exp/FLT_M23_infer_expt.out

Reading reference gene model /content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/References/Mus_musculus.GRCm39.112.gtf.bed ... Done
Loading SAM/BAM file ...  Finished
Total 150342 usable reads were sampled


<div class="alert alert-block alert-info">
<b><i>Code Breakdown</i></b>
    
<br>

**Parameter Definitions:**

* `-r` – specifies the path to the reference gene model in BED format
* `-i` – specifies the path to the input alignment file (in SAM or BAM format)
* `-s` – specifies the number of reads to sample from the input alignment file
  > _Note: Since our samples only contains ~0.1M paired-end reads and 15M total reads are specified, all sample reads will be evaluated._
* `> $RSeQC_infer_exp/FLT_M23_infer_expt.out` – redirects the standard output to a file ending in `*infer_expt.out`

**Input Data:**
- `Mus_musculus.GRCm39.112.gtf.bed` (mouse reference gene model in BED format, output from [Step 3](#3.-Create-a-BED-File-for-the-Reference-Genome))
- `*Aligned.sortedByCoord.out.bam` (binary, alignment map containing sorted reads mapping to the genome, output from [Step 2a](#2a.-Align-Trimmed-Sequence-Data-with-STAR))

**Output Data:**
- `*infer_expt.out` (quantitation of read strandedness relative to the strandedness of reference transcripts)

</div>

<br>___________

Let's list the output file that was generated:

In [26]:
!ls -1 $RSeQC_infer_exp/FLT_M23_infer_expt.out

/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/RSeQC_analyses/infer_exp/FLT_M23_infer_expt.out


<br>

<div class="alert alert-block alert-info">
<b>Note:</b><br>
    
To preserve compute resources, the strandedness has been determined for all OSD-104 FLT and GC samples using the same command.

</div>


<br>

---

Next, we'll use the cat command to view the standard output from the RSeQC infer experiment command:


In [27]:
!cat $RSeQC_infer_exp/FLT_M23_infer_expt.out



This is PairEnd Data
Fraction of reads failed to determine: 0.1550
Fraction of reads explained by "1++,1--,2+-,2-+": 0.0103
Fraction of reads explained by "1+-,1-+,2++,2--": 0.8347


<div class="alert alert-block alert-info">
<b>RSeQC Infer Experiment output content</b><br>

The standard output of the [RSeQC Infer Experiment program](http://rseqc.sourceforge.net/#infer-experiment-py) contains the following info:

- Indication of paired-end (PE) or single-end (SE) data


- Fraction of reads whose strandedness is undetermined


- For SE data:
  - Fraction of reads explained by "++,--" (reads are the same orientation relative to the reference)
  - Fraction of reads explained by "+-,-+" (reads are the reverse compliment orientation relative to the reference)


- For PE data:
  - Fraction of reads explained by "1++,1--,2+-,2-+" (forward reads are the same orientation relative to the reference and reverse reads are the reverse compliment)
  - Fraction of reads explained by "1+-,1-+,2++,2--" (forward reads are the reverse compliment orientation relative to the reference and reverse reads are the same orientation)

</div>

<br>

**Look at the `*infer_exp.out` file above and answer the following questions:**

1. What percent of reads have undetermined strandedness?

2. What percent of reads are sense relative to the reference?

3. What percent of reads are antisense relative to the reference?

4. Are these data stranded?

**Input your responses to each question in the text boxes below:**
> *Answer each question in your own words BEFORE checking your answers against the solutions below. Please do not copy and paste the answers provided.*


Question 1:
Fraction of reads failed to determine: 0.1550, or 15.50%

Question 2: 1.03%,  because reads with "1++,1--,2+-,2-+" are forward reads with same orientation with respect to the reference


Question 3: 83.47%


Question 4:
yes in the antisense verse


<br>

<div class="alert alert-block alert-success">

<details>
<summary><b>Q1 Solution</b></summary>

<br>

15.50% of reads have undetermined strandedness.

</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q2 Solution</b></summary>

<br>

1.03% of reads are sense relative to the reference.
    
</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q3 Solution</b></summary>

<br>

83.47% of reads are antisense relative to the reference.

</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q4 Solution</b></summary>

<br>

Yes, these data are stranded in the antisense direction.

</details>
</div>


<br>

---


<a class="anchor" id="inferexpmultiqc"></a>

### 4b. Compile Strandedness Data with MultiQC

Rather than viewing the `*infer_exp.out` file for all 12 aligned samples individually, we'll use the [MultiQC](https://docs.seqera.io/multiqc) program to compile the strandedness data from all samples and generate a single report (and associated data files). This will allow us to view the strandedness of all samples at once.

Let's compile the data in the `*infer_exp.out` files from our 12 samples using MultiQC by running the following command:

In [30]:
!multiqc --interactive -n infer_exp_multiqc -o $RSeQC_infer_exp_multiqc $RSeQC_infer_exp/


/// ]8;id=589768;https://multiqc.info\MultiQC]8;;\ 🎄 v1.33

       file_search | Search path: /content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/RSeQC_analyses/infer_exp
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 21/21  
             rseqc | Found 12 infer_experiment reports
      exec_modules | Updating module "RSeQC"
     write_results | Existing reports found, adding suffix to filenames. Use '--force' to overwrite.
     write_results | Data        : mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/RSeQC_analyses/infer_exp/infer_exp_multiqc_report/infer_exp_multiqc_data_1
     write_results | Report      : mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/RSeQC_analyses/infer_exp/infer_exp_multiqc_report/infer_exp_multiqc_1.html
           multiqc | MultiQC complete


<div class="alert alert-block alert-info">
<b><i>Code Breakdown</i></b>
    
<br>

**Parameter Definitions:**

* `--interactive` – force reports to use interactive plots
* `-n` – prefix name for the output files
* `-o` – the output directory to store the MultiQC results
* `$RSeQC_infer_exp/` – the directory holding the RSeQC Infer Experiment output file from each sample, provided as a positional argument

**Input Data:**
- `*infer_exp.out` (quantitation of read strandedness relative to the strandedness of reference transcripts)

**Output Data:**
- `infer_exp_multiqc_data` (directory containing the compiled strandedness data from each sample, used to create the MultiQC report)
- `infer_exp_multiqc.html` (MultiQC report: an interactive webpage file containing graphical representations of the compiled strandedness data)

</div>

<br>___________


MultiQC is finished running when the loading animation on the left end of the code cell stops and a green checkmark appears. In a standard bash shell environment, if the multiQC job finishes successfully, you will see a "MultiQC complete" statement at the end of the standard output, and could check that the number of reports found matches what you expect.
> Note: There is only one `*infer_exp.out` file generated per sample (for paired- or single-end data), so we expect one report per sample. Since we have 12 samples, MultiQC should detect 12 RSeQC infer exp reports.

Let's list the output files that were generated:


In [31]:
!ls -1 $RSeQC_infer_exp_multiqc/*

/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/RSeQC_analyses/infer_exp/infer_exp_multiqc_report/infer_exp_multiqc_1.html
/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/RSeQC_analyses/infer_exp/infer_exp_multiqc_report/infer_exp_multiqc.html

/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/RSeQC_analyses/infer_exp/infer_exp_multiqc_report/infer_exp_multiqc_data:
llms-full.txt
multiqc_citations.txt
multiqc_data.json
multiqc.log
multiqc.parquet
multiqc_rseqc_infer_experiment.txt
multiqc_sources.txt
rseqc_infer_experiment_plot.txt

/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/RSeQC_analyses/infer_exp/infer_exp_multiqc_report/infer_exp_multiqc_data_1:
llms-full.txt
multiqc_citations.txt
multiqc_data.json
multiqc.log
multiqc.parquet
multiqc_rseqc_infer_experiment.txt
multiqc_sources.txt
rseqc_infer_experiment_plot.txt


Open the `infer_exp_multiqc_report` directory in the side panel of this Colab notebook by clicking: `/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/RSeQC_analyses/infer_exp/infer_exp_multiqc_report`, as shown below.
> *Note: These files are stored in your Google Drive folder.*

<br>

<div align="center">
<img src="https://raw.githubusercontent.com/nasa/GeneLab-Training/refs/heads/GL4U_RNAseq_2024_Colab/images/gd_infer_exp_multiqc_dir.png" alt="Open the infer experiment multiQC report directory">
</div>
<br>

Click the 3 dots next to the `infer_exp_multiqc.html` file in the left side panel to download it to your computer as shown below.

<br>

<div align="center">
<img src="https://raw.githubusercontent.com/nasa/GeneLab-Training/refs/heads/GL4U_RNAseq_2024_Colab/images/gd_dl_infer_exp_multiqc.png" alt="Save JN">
</div>
<br>

Navigate to the HTML file you downloaded on your computer then double click on it to open in your web browser to view the compiled strandedness info from all 12 samples, which should look similar to the following:

<br>

<div align="center">
<img src="https://raw.githubusercontent.com/nasa/GeneLab-Training/refs/heads/GL4U_RNAseq_2024_Colab/images/gd_open_infer_exp_multiqc.png" alt="Open strandedness multiQC report">
</div>
<br>
<div class="alert alert-block alert-info">
<b>Note:</b><br>
    
Click [here](https://docs.seqera.io/multiqc/modules/rseqc) for a description of the MultiQC plots derived from RSeQC data.

</div>

<div class="alert alert-block alert-warning">
    
Be sure to click the "Trust HTML" button in the top left corner of the MultiQC report to be able to see all the plots.

If the plots are still not visible, make sure JavaScript is enabled on the web-browser you are using.

</div>

<br>
---

**Take a look at the MultiQC report of the strandedness data and answer the following questions:**

1. Are the data for all samples stranded or unstranded?

2. If stranded, what is the orientation of the reads relative to the reference?

**Input your responses to each question in the text boxes below:**
> *Answer each question in your own words BEFORE checking your answers against the solutions below. Please do not copy and paste the answers provided.*


Question 1: all samples are stranded

Question 2:
 the orientation is antisense for all the samples

<br>

<div class="alert alert-block alert-success">

<details>
<summary><b>Q1 Solution</b></summary>

<br>

All samples are stranded.

</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q2 Solution</b></summary>

<br>

All samples are stranded in the antisense orientation relative to the reference.

</details>
</div>


<br>

---


<a class="anchor" id="rsemindex"></a>

## 5. Build a RSEM Index for the Reference Genome

The next step is to quantify the number of reads that align to each transcript with the [RSEM](https://deweylab.github.io/RSEM/) (RNA-Seq by Expectation-Maximization) transcript quantification method.

In order to quantify the aligned reads in the BAM file generated during alignment, the RSEM program first requires the generation of an RSEM index using the reference genome and associated GTF files from the sample organism (similar to STAR). To do this, we will first run the RSEM `rsem-prepare-reference` program, which will extract reference transcripts from the mouse genome using the gene transfer file to create a RSEM indexed reference.  

<div class="alert alert-block alert-info">
<b>Note:</b><br>

To save time and storage space, we created the RSEM index for you. You're welcome :0)

</div>

The following command was used to generate the RSEM index for _Mus musculus_:

```bash
rsem-prepare-reference --gtf Mus_musculus.GRCm39.112.gtf \
 Mus_musculus.GRCm39.dna.primary_assembly.fa \
 RSEM_index
```


<div class="alert alert-block alert-info">
<b><i>Code Breakdown</i></b>
    
<br>

**Parameter Definitions:**

* `--gtf` – specifies the uncompressed file containing annotated transcripts in the standard GTF format
* `Mus_musculus.GRCm39.dna.primary_assembly.fa` – specifies the uncompressed fasta file containing the genome reference sequences, provided as a positional argument
* `RSEM_index` – specifies the path to the directory where the RSEM reference will be stored and the prefix desired for the RSEM reference files, provided as a positional argument

**Input Data:**
- `*.fasta` (genome sequence)
- `*.gtf` (genome transfer feature file)

**Output Data:**

RSEM indexed genome reference, which consists of the following files:
- `Mmus.chrlist`
- `Mmus.grp`
- `Mmus.idx.fa`
- `Mmus.n2g.idx.fa`
- `Mmus.seq`
- `Mmus.ti`
- `Mmus.transcripts.fa`

</div>

---

<a class="anchor" id="count"></a>

## 6. Quantitate Alignment Data

<a class="anchor" id="rsemcount"></a>

### 6a. Count Aligned Reads with RSEM

Now we're ready to quantify the aligned reads with RSEM. Similar to read alignment, quantifying the number of reads that aligned to each gene can also be quite challenging. How so?

**Input your response to the question in the text box below:**

<br>

<div class="alert alert-block alert-success">

<details>
<summary><b>Answer</b></summary>

<br>

If a read maps to more than one location on the genome, how do we know from which location it truly originated? It is also difficult to quantify alternative splice isoforms of a gene with significant sequence overlap.

</details>
</div>

<br>

The RSEM software package was designed to address these issues and "rescue" reads that are not uniquely mapped by allocating them to transcripts using a 3-step algorithm:

 1. Estimate abundances based on uniquely mapped reads only.
 2. For each read that maps to multiple locations (multiread), divide it between the transcripts to which it maps, proportional to their abundances estimated in the first step.
 3. Compute abundances based on updated counts for each transcript.

Therefore, we will use RSEM to quantify the aligned reads, due to its ability to account for reads that map to multiple transcripts and distinguish gene isoforms.

<div class="alert alert-block alert-info">
<b>Note:</b><br>

Quantitating alignment data with RSEM requires a lot of compute and storage resources and can take a while to complete, so we did it for you. You're welcome :0)

The RSEM quantitation files we are using were generated using all reads of each sample from [OSD-104](https://osdr.nasa.gov/bio/repo/data/studies/OSD-104).

</div>


The following `RSEM` command was used to estimate gene and isoform expression levels for all 12 samples:
> Full information on this command is available in the [RSEM user guide](http://deweylab.biostat.wisc.edu/rsem/rsem-calculate-expression.html).

```bash
rsem-calculate-expression --num-threads 13 \
 --alignments \
 --bam \
 --paired-end \
 --seed 12345 \
 --seed-length 20 \
 --estimate-rspd \
 --no-bam-output \
 --strandedness reverse \
 $STAR_output/${sample}/${sample}_Aligned.toTranscriptome.out.bam \
 RSEM_index \
 $RSEM_output/${sample}
```

<div class="alert alert-block alert-info">
<b><i>Code Breakdown</i></b>
    
<br>

**Parameter Definitions:**

* `--num-threads` – specifies the number of threads to use
* `--alignments` - indicates that the input file contains alignments in sam, bam, or cram format
* `--bam` - specifies that the input alignments are in bam format
* `--paired-end` - indicates that the input reads are paired-end reads
* `--seed` - the seed for the random number generators used in calculating posterior mean estimates and credibility intervals; must be a non-negative 32-bit integer
* `--seed-length` - instructs RSEM to ignore any aligned read if it or its mate's (for paired-end reads) length is less than the value indicated (20bp, in the command above)
* `--estimate-rspd` - instructs RSEM to estimate the read start position distribution (rspd) from the data
* `--no-bam-output` - instructs RSEM not to output any bam file
* `--strandedness` - defines the strandedness of the RNAseq reads; the `reverse` option means reads are antisense relative to the reference, indicated from the [Step 4](#4.-Determine-Read-Strandedness) output files
* `$STAR_output/${sample}/${sample}_Aligned.toTranscriptome.out.bam` - specifies path to input bam file(s), provided as a positional argument
* `RSEM_index` - specifies the path to the directory where the RSEM reference is stored and its prefix, provided as a positional argument
* `$RSEM_output/${sample}` – specifies the path to and prefix for the output file names, provided as a positional argument; for GeneLab the prefix is the sample id

**Input Data:**
- RSEM indexed genome reference (output from [Step 5](#5.-Build-a-RSEM-Index-for-the-Reference-Genome))
- `*Aligned.toTranscriptome.out.bam` (binary, alignment map containing sorted reads mapping to transcriptome, output from [Step 2a](#2a.-Align-Trimmed-Sequence-Data-with-STAR))

**Output Data:**
- `*genes.results` (expression estimates per gene)
- `*isoforms.results` (expression estimates per isoform)
- `*stat` (directory containing the following stats files)
	- `*cnt`
	- `*model`
	- `*theta`

</div>

___

Let's take a look at the first few lines of the `*genes.results` output file from sample FLT_M23 using the following command:

In [ ]:
!head $RSEM_output/FLT_M23.genes.results

<div class="alert alert-block alert-info">

<b>RSEM `*genes.results` content</b>


The primary output of RSEM consists of two files, one for isoform-level estimates (`*isoforms.results`), and the other for gene-level estimates (`*genes.results`). Each file consists of gene and transcript ids and lengths, and abundance estimates given in different measures. In the `*genes.results` files, you'll find the following tab-separated fields in order of column location:

1) `gene_id`: Gene name according to the database used (in this example, we used ensembl genome and GTF files, so the gene names are ensembl IDs).

2) `transcript_id(s)`: Comma-separated list of all the transcripts derived from the respective gene in column 1.

3) `length`: The weighted average of the respective gene's transcripts' lengths.

4) `effective_length`: The weighted average of the respective gene's transcripts' effective lengths, which are weighted by each transcript's isoform percentage.

5) `expected_count`: The sum of the estimates of the number of read fragments that are derived from each transcript of the respective gene (we will use these count values to calculate differential expression).

6) `TPM`: Transcripts per million, which is a relative measure of transcript abundance - this value is summed over all transcripts for each respective gene to generate the gene TPM value.

7) `FPKM`: Fragments Per Kilobase of transcript per Million mapped reads, which is another relative measure of transcript abundance - this value is summed over all transcripts for each respective gene to generate the gene FPKM value.


> _Detailed descriptions of all RSEM output files can be found [here](http://deweylab.biostat.wisc.edu/rsem/rsem-calculate-expression.html#output)._

</div>


**Now that you know what the fields mean, look at the `FLT_M23.genes.results` file above and answer the following questions:**

1. What is the first gene listed?

2. For that first gene,

   a. What is the gene length?
   
   b. What is the expected count?
   
   c. What is the TPM value?  
    
3. What are those values for the third gene listed?

**Input your responses to each question in the text boxes below:**
> *Answer each question in your own words BEFORE checking your answers against the solutions below. Please do not copy and paste the answers provided.*


Question 1:


Question 2:


Question 3:


<br>

<div class="alert alert-block alert-success">

<details>
<summary><b>Q1 Solution</b></summary>

<br>

The first gene listed is: ENSMUSG00000000001

</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q2 Solution</b></summary>

<br>

Gene length: 3262.00 bp
Expected count: 869.00
TPM value: 5.93
    
</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q3 Solution</b></summary>

<br>

The third gene listed is: ENSMUSG00000000028
Gene length: 1860.88 bp
Expected count: 91.00
TPM value: 1.15

</details>
</div>

<br>

---


**Challenge:**

Take a look at the first few lines of the `*isoforms.results` file for the same sample as above using the following command:

In [ ]:
!head $RSEM_output/FLT_M23.isoforms.results

___

**Look at the `FLT_M23.isoforms.results` file above and answer the following questions:**

1. Calculate the sum of the expected count values for all isoforms associated with this gene: ENSMUSG00000000028.

2. How does that value compare with the expected count for gene ENSMUSG00000000028 in the `FLT_M23.genes.results` file?

**Input your responses to each question in the text boxes below:**
> *Answer each question in your own words BEFORE checking your answers against the solutions below. Please do not copy and paste the answers provided.*


Question 1:


Question 2:


<br>

<div class="alert alert-block alert-success">

<details>
<summary><b>Q1 Solution</b></summary>

<br>

The sum of the expected count values for all isoforms associated with ENSMUSG00000000028 is:
74.39 (ENSMUST00000000028) + 10.45 (ENSMUST00000096990) + 6.17 (ENSMUST00000115585) + 0.00 (ENSMUST00000231819) = 91.01

</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q2 Solution</b></summary>

<br>

It is the same. This is because the expected count value for each gene is the sum of the expected count values for each isoform of that gene.
    
</details>
</div>


---

<a class="anchor" id="rsemmultiqc"></a>

### 6b. Compile Count Data with MultiQC

Let's take a look at the stats for the RSEM count step but rather than viewing the stats for each sample individually, we'll use the [MultiQC](https://docs.seqera.io/multiqc) program to compile the RSEM `*.cnt` stats file from all samples and generate a single report (and associated data files). This will allow us to view the quantitation statistics of all samples at once.

Let's compile the data in the `*.cnt` stats files from our 12 samples using MultiQC by running the following command:

In [ ]:
!multiqc --interactive -n RSEM_count_multiqc -o $count_multiqc $RSEM_logs/

<div class="alert alert-block alert-info">
<b><i>Code Breakdown</i></b>
    
<br>

**Parameter Definitions:**

* `--interactive` – force reports to use interactive plots
* `-n` – prefix name for the output files
* `-o` – the output directory to store the MultiQC results
* `$RSEM_logs/` – the directory holding the RSEM output files including the *.cnt stats file from each sample, provided as a positional argument

**Input Data:**
- `*.cnt` (summary alignment info/stats data used for quantitation, output from Step 6a)

**Output Data:**
- `count_multiqc_data` (directory containing the compiled summary of the quantifiable alignment info/stats data from each sample, used to create the MultiQC report)
- `count_multiqc.html` (MultiQC report: an interactive webpage file containing graphical representations of the compiled quantifiable alignment info/stats data)

</div>

<br>
___________



MultiQC is finished running when the loading animation on the left end of the code cell stops and a green checkmark appears. In a standard bash shell environment, if the multiQC job finishes successfully, you will see a "MultiQC complete" statement at the end of the standard output, and could check that the number of reports found matches what you expect.
> Note: There is only one `*.cnt` file generated per sample (for paired- or single-end data), so we expect one report per sample. Since we have 12 samples, MultiQC should detect 12 RSEM reports.

Let's list the output files that were generated:

In [ ]:
!ls -1 $count_multiqc/*

Open the `count_multiqc_report` directory in the side panel of this Colab notebook by clicking: `/content/mnt/MyDrive/NASA/GL4U/RNAseq/OSD-104/03-RSEM_Counts/count_multiqc_report`, as shown below.
> *Note: These files are stored in your Google Drive folder.*

<br>

<div align="center">
<img src="https://raw.githubusercontent.com/nasa/GeneLab-Training/refs/heads/GL4U_RNAseq_2024_Colab/images/gd_rsem_multiqc_dir.png" alt="Open the rsem multiQC report directory">
</div>
<br>

Click the 3 dots next to the `RSEM_count_multiqc.html` file in the left side panel to download it to your computer as shown below.

<br>

<div align="center">
<img src="https://raw.githubusercontent.com/nasa/GeneLab-Training/refs/heads/GL4U_RNAseq_2024_Colab/images/gd_dl_rsem_multiqc.png" alt="Download rsem multiqc HTML file">
</div>
<br>

Navigate to the HTML file you downloaded on your computer then double click on it to open in your web browser to view the compiled quantitation info from all 12 samples, which should look similar to the following:

<br>

<div align="center">
<img src="https://raw.githubusercontent.com/nasa/GeneLab-Training/refs/heads/GL4U_RNAseq_2024_Colab/images/gd_open_rsem_multiqc.png" alt="Open rsem multiQC report">
</div>
<br>
<div class="alert alert-block alert-info">
<b>Note:</b><br>
    
Click [here](https://docs.seqera.io/multiqc/modules/rsem) for a description of the MultiQC plots derived from RSEM data.

</div>

<div class="alert alert-block alert-warning">
    
Be sure to click the "Trust HTML" button in the top left corner of the MultiQC report to be able to see all the plots.

If the plots are still not visible, make sure JavaScript is enabled on the web-browser you are using.

</div>

---

**Take a look at the MultiQC report of the RSEM `*.cnt` data above and answer the following questions:**

1. Which sample has the highest % alignable reads? Which sample has the lowest?

2. Which sample has the highest % of unalignable reads? Which sample has the lowest?

**Input your responses to each question in the text boxes below:**
> *Answer each question in your own words BEFORE checking your answers against the solutions below. Please do not copy and paste the answers provided.*


Question 1:


Question 2:


<br>

<div class="alert alert-block alert-success">

<details>
<summary><b>Q1 Solution</b></summary>

<br>

Sample with the highest % alignable reads: GC_M34 (90.5%)
Sample with the lowest % alignable reads: GC_M38 (87.0%)

</details>
</div>


<div class="alert alert-block alert-success">

<details>
<summary><b>Q2 Solution</b></summary>

<br>

Sample with the highest % unalignable reads: GC_M38 (13.0%)
Sample with the lowest % unalignable reads: GC_M34 (9.5%)

</details>
</div>


<br>

---

We have now generated RSEM raw counts for all genes for all 12 samples in our study (OSD-104). We will use the RSEM `*genes.results` files in the next notebook to perform downstream data analyses including differential gene expression (DGE), principal component analysis (PCA), gene set enrichment analysis (GSEA), and generate respective visualizations to help interpret the results of our analyses.



<a class="anchor" id="copy-to-gd"></a>

## 7. Copy Completed JN to Google Drive
Double check all sections in this JN to make sure that all activities have been completed. Then, save a copy of your completed JN to your Google Drive by selecting "Save a copy in Drive" under "File" in the top left corner of this JN as shown below.

<br>

<div align="center">
<img src="https://raw.githubusercontent.com/nasa/GeneLab-Training/refs/heads/GL4U_RNAseq_2024_Colab/images/save_RNAseq_02_JN.png" alt="Save JN">
</div>

<br>
After successfully saving your completed JN to your Google Drive, open the <a href="https://github.com/nasa/GeneLab-Training/blob/GL4U_RNAseq_2024_Colab/On-Demand/Access_Processing_2of2_JN.md/#upload-your-completed-jn-to-canvas">Access_Processing_2of2_JN</a> instructions in a new tab, and follow the steps in the "Upload Your Completed JN To Canvas" section to download your completed JN then upload it to Canvas to receive credit.
<br>